<b>Project - Helpline of all sorts</b><br><br>
Implimenting LSTM model to predict what the concern is about:<br>
0 - work life harassment and stress<br>
1 - school bullying related complaints<br>
2 - sexual harassment related complaints<br>
3 - whistleblowing<br>

In [1]:
import numpy

numpy.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
numpy.random.seed(1)

<br><br><b>Converting input sentences into vector word representation, leveraged pre-trained 100-D GloVe embedding.</b>

In [6]:
import os
data_dir = 'glove.6B.100d.txt'

embeddings_index = {}
word_to_index = {}
index_to_word = {}
# f = open(os.path.join(data_dir, 'glove.6B.100d.txt'))
f = open(data_dir, encoding="utf8")
i = 0
for line in f:
    values = line.split()
    word = values[0]
    word_to_index[word] = i
    index_to_word[i] = word
    embedding = numpy.asarray(values[1:], dtype='float32')
    embeddings_index[word] = embedding
    i+=1
f.close()

# for testing purpose to make sure we are getting correct word vectors
print('Found %s word vectors.' % len(embeddings_index))
print("sample embedding of word 'bangalore' is", embeddings_index['bangalore'])
word = "cucumber"
index = 50000
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index+1) + "th word in the vocabulary is", index_to_word[index])

Found 400000 word vectors.
sample embedding of word 'bangalore' is [-0.99448  -0.077857  0.7501    0.1678    0.27551   0.020159 -0.50652
  0.8895   -0.87816   1.0889   -0.067507 -0.49307   0.32534   0.50727
  0.15708  -0.71007   0.3012    0.92905  -0.26814   0.59854   0.19417
 -0.51151   0.32488   0.40267  -0.14587   0.071352  0.64874   0.13758
  0.007469 -0.18785  -0.76551   0.30816   0.36882  -0.41252  -0.2235
 -0.29064  -0.92269   0.39056  -0.35599   0.070162  0.033666  0.32743
 -0.14269  -0.65097   0.31394  -0.044538  0.95459   0.39427   0.40993
  1.2348   -0.85968  -0.072344  1.0908   -0.1989   -0.40554  -0.56707
 -1.2005    0.20608   0.28379  -0.44239  -0.57755  -0.28433  -0.42684
  0.77296  -0.82195   0.83016  -0.36525  -0.18728   0.42421   0.73417
 -0.21351   0.2617   -0.091433 -1.0071   -0.81744  -0.31586   0.75839
 -0.061132 -0.74389  -0.053195  0.21087  -0.26775  -0.40766   0.67072
 -0.78855   0.074657 -0.32326  -0.034044 -0.50166   0.54098   0.51638
 -0.063815  0.19218  -0.

<br><br><b>loading the training datsaset</b>

In [13]:
import csv

filename = 'helpline_datasets.csv'
X_train_list = []
Y_train_list = []
with open(filename,'r',encoding="utf-8") as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        X_train_list.append(row[0])
        Y_train_list.append(row[1])
            
X_train = numpy.array(X_train_list) # converting list into 1-D vector representation   
Y_train = numpy.array(Y_train_list) # converting list into 1-D vector representation
num_training_sets = 0
try:
    num_training_sets = numpy.prod(X_train.shape)
    print("total datasets in training: ",num_training_sets)
    if num_training_sets != numpy.prod(X_train.shape):
        raise ValueError
except ValueError:
    print("dimensions of datasets won't match, verify once!!!!!")
maxLen = len(max(X_train, key=len).split())
index = 9
print("sample of training set in 9th index: ",X_train[index], Y_train[index])

total datasets in training:  29
sample of training set in 9th index:  As a librarian, I've been threatened with stalking for not giving my phone number to a patron. I've also been shown naked, sexually explicit photos; and propositioned by countless men, some of which were married and standing right next to their children. Some of my female colleagues have had male patrons waiting for them in the parking lot after work, and they've received explicit messages over social media. 2


<br><br><b>Converting array of sentences into array of indices so to feed embeddings() function</b>

In [14]:
def sentences_to_indices(X, word_to_index, max_len):
    m = X.shape[0] #number of training examples
    X_indices = numpy.zeros((m, max_len))
    for i in range(m):
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = X[i].lower().split()
        j = 0
        for w in sentence_words:
            X_indices[i,j] = word_to_index[w]
            j = j+1
    return X_indices

In [15]:
X1 = numpy.array(["for the past several days I'm suffering from a lot of mental tension due to ongoing audits in office late night",
                 "I got bullied yesterday from a bunch of senior guys during interval"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 21)
print("X1=",X1)
print("X1_indices=",X1_indices)

KeyError: "i'm"